In [1]:
include("../src/KagomeTransport.jl");
using .KagomeTransport.Kagome_Hamiltonian;
using LinearAlgebra;
using StaticArrays;


LoadError: LoadError: ParseError:
# Error @ /home/nabil/work/projects/kagome/KagomeTransport/src/HamiltonianConstructors.jl:289:5

end 
#   └ ── Expected `end`
in expression starting at /home/nabil/work/projects/kagome/KagomeTransport/src/HamiltonianConstructors.jl:289
in expression starting at /home/nabil/work/projects/kagome/KagomeTransport/src/KagomeTransport.jl:1

In [2]:
"""
Calculate the xy comp. of the complex quantum metric (both the real quantum geometry and berry curvature of the bands at a single point in the BZ)
"""
@inline function Complex_Metric_xy(;
    E::Vector{Float64}, 
    U::Matrix{ComplexF64}, 
    Vxb::Matrix{ComplexF64}, 
    Vyb::Matrix{ComplexF64},
    Num_Bands::Int64)
    
    ωmn = E' .- E
    ωmn[ωmn .== 0] .= 1e-15
    
    setindex!.(Ref(Vxb), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)
    setindex!.(Ref(Vyb), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)

    Xmn_x = -im * (Vxb ./ ωmn); Xmn_y = -im * (Vyb ./ ωmn);
    Xmn_x * Xmn_y'
end


@inline function Complex_Metric_xy(;
    k::Vector{Float64}, p::Params, Num_Bands::Int64 = 3)

    (E, U) = eigensystem3c(k, p)
    ωmn = E' .- E
    ωmn[ωmn .== 0] .= 1e-15

    Velx_band::Matrix{Float64} = U' * Velx(k[1], k[2], p) * U
    Vely_band::Matrix{Float64} = U' * Vely(k[1], k[2], p) * U
    
    setindex!.(Ref(Velx_band), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)
    setindex!.(Ref(Vely_band), 0.0 + im * 0.0, 1:Num_Bands, 1:Num_Bands)

    Xmn_x = -im * (Velx_band ./ ωmn); Xmn_y = -im * (Vely_band ./ ωmn);
    Xmn_x * Xmn_y'
end




UndefVarError: UndefVarError: `Params` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [3]:
# Preliminary calculation of the Berry curvature 
let npts_perdim = 300, lim = 1.0
    
    # construction of the grid 
    rng = LinRange(-lim * pi, lim * pi, npts_perdim)
    grid = Matrix{Vector{Float64}}(undef, npts_perdim, npts_perdim)
    for (n, kx) ∈ enumerate(rng)
        for (m, ky) ∈ enumerate(rng)
            grid[n, m] = [kx, ky]
        end
    end    
    res = Matrix{Vector{Float64}}(undef, npts_perdim, npts_perdim)
    res .= Complex_Metric_xy(k = grid, p = Ref(p))
end


UndefVarError: UndefVarError: `p` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [4]:
p = Params()
Complex_Metric_xy(k = [0.0, 0.0], p = p , Num_Bands = 3)

UndefVarError: UndefVarError: `Params` not defined in `Main`
Suggestion: check for spelling errors or missing imports.